CS494 - Colab 3
## Studying COVID-19

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Now we authenticate a Google Drive client to download the files we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
# Updated 4/22
id='1jOcwW4RWnOvZl563-L2xJF0RiAA6lryf'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_19-covid-Confirmed.csv')

id='1xvbVjr22XbKUXv_Lhts8MOSpFc_825Ko'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_19-covid-Deaths.csv')

id='1Xktj9-B4yCaJyKKWnxnwAzYQ9eFhRR50'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('time_series_19-covid-Recovered.csv')

If you executed the cells above, you should be able to see the dataset we will use for this Colab under the "Files" tab on the left panel.

Next, we import some of the common libraries needed for our task.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [ ]:
spark

In [ ]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

If you are running this Colab on the Google hosted runtime, the cell below will create a *ngrok* tunnel which will allow you to still check the Spark UI.

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Data Loading

# New Section

In this Colab, we will be analyzing the timeseries data of the Coronavirus COVID-19 Global Cases, collected by Johns Hopkins CSSE.

Here you can check a realtime dashboard based on this dataset: [https://www.arcgis.com/apps/opsdashboard/index.html?fbclid=IwAR2hQKsEZ3D38wVtXGryUhP9CG0Z6MYbUM_boPEaV8FBe71wUvDPc65ZG78#/bda7594740fd40299423467b48e9ecf6](https://www.arcgis.com/apps/opsdashboard/index.html?fbclid=IwAR2hQKsEZ3D38wVtXGryUhP9CG0Z6MYbUM_boPEaV8FBe71wUvDPc65ZG78#/bda7594740fd40299423467b48e9ecf6)

---



*   ```confirmed```: dataframe containing the total number of confirmed COVID-19 cases per day, divided by geographical area
*   ```deaths```: dataframe containing the number of deaths per day due to COVID-19, divided by geographical area
*   ```recovered```: dataframe containing the number of recoevered patients per day, divided by geographical area







In [ ]:
confirmed = spark.read.csv('time_series_19-covid-Confirmed.csv', header=True)
deaths = spark.read.csv('time_series_19-covid-Deaths.csv', header=True)
recovered = spark.read.csv('time_series_19-covid-Recovered.csv', header=True)

In [ ]:
confirmed.show()

### Your Task

We are aware of the great deal of stress we are all experiencing because of the spread of the Coronavirus. As such, we decided to conclude our series of Colabs with a **lightweight task** -- given everything you have learned about Spark during the quarter, this Colab should take you just a few minutes to complete.

At the same time, we turned this into an opportunity to raise awareness about the extent of the COVID-19 epidemic.

[Stay healthy, wash your hands often](https://www.cdc.gov/coronavirus/2019-ncov/about/index.html), and invest the time you saved on this Colab to be emotionally supportive to your friends and family!

Signed, *the CS494 teaching staff*

In [ ]:
# YOUR CODE HERE
#this code counts how many confirmed, death and recovered are in the last update 
countConfirmed = confirmed.agg(sum("4/22/20").alias("Confirmed"))
countDeaths = deaths.agg(sum("4/22/20").alias("Deaths"))
countRecovered = recovered.agg(sum("4/22/20").alias("Recovered"))

countConfirmed.show()
countDeaths.show()
countRecovered.show()

+---------+
|Confirmed|
+---------+
|2624089.0|
+---------+

+--------+
|  Deaths|
+--------+
|183064.0|
+--------+

+---------+
|Recovered|
+---------+
| 709694.0|
+---------+



Consider only the most recent data points in the timeseries, and compute:


*   number of confirmed COVID-19 cases across the globe
*   number of deaths due to COVID-19 across the globe
*   number of recovered patients across the globe



In [ ]:
# YOUR CODE HERE

from pyspark.sql.types import IntegerType

#confirmed less than 50 cases
cases = confirmed.withColumn("4/22/20", confirmed["4/22/20"].cast(IntegerType()))
sortConfirmed = cases.groupby("Country/Region").sum("4/22/20")
cases = sortConfirmed.filter(sortConfirmed["sum(4/22/20)"] >= 50)
cases.show(10)
casesByCountry = [str(row['Country/Region']) for row in cases.collect()]
confirmedCount = cases.agg(sum("sum(4/22/20)").alias("Confirmed"))
confirmedCount.show()

#recovered cases
casesRecovered = recovered.filter(col("Country/Region").isin(casesByCountry))
countRecovered = casesRecovered.agg(sum("4/22/20").alias("Recovered"))
countRecovered.show()
totalRecovered = countRecovered.select("*").toPandas()
confirmedCount = confirmedCount.select("*").toPandas()

#ratio between confirmed and recovered
ratio = totalRecovered["Recovered"][0]/confirmedCount["Confirmed"][0]
ratio 



+--------------+------------+
|Country/Region|sum(4/22/20)|
+--------------+------------+
|      Paraguay|         213|
|        Russia|       57999|
|       Senegal|         442|
|    Cabo Verde|          73|
|        Sweden|       16004|
|        Guyana|          67|
|   Philippines|        6710|
|         Burma|         123|
|      Djibouti|         974|
|      Malaysia|        5532|
+--------------+------------+
only showing top 10 rows

+---------+
|Confirmed|
+---------+
|  2623413|
+---------+

+---------+
|Recovered|
+---------+
| 709523.0|
+---------+



0.27045798736226434

Consider only the most recent data points in the timeseries, and filter out the geographical locations where less than 50 cases have been confirmed.
For the areas still taken into consideration after the filtering step, compute the ratio between number of recovered patients and number of confirmed cases.

In [ ]:
# YOUR CODE HERE

#number of deaths 
deathCount = deaths.filter(col("Country/Region").isin(casesByCountry))
totalDeaths = deathCount.agg(sum("4/22/20").alias("Deaths"))
totalDeaths.show()
totalDeaths = totalDeaths.select("*").toPandas()
#ratio between number of deaths and number of confirmed cases
ratio = totalDeaths["Deaths"][0]/confirmedCount["Confirmed"][0]
ratio 


+--------+
|  Deaths|
+--------+
|183037.0|
+--------+



0.06977056224086714

Following the same filtering strategy as above, now compute the ratio between number of deaths and number of confirmed cases.

**Optional:** Create a global map of the ratio between recovered / confirmed, recovered / deaths, and confirmed / deaths in the final data points in the timeseries using a mapping tool such as Leaflet.js (https://leafletjs.com/).

In [ ]:
# YOUR CODE HERE (Optional)

Once you have working code for each cell above, **head over to Gradescope, read carefully the questions, and submit your solution for this Colab**!